<a href="https://colab.research.google.com/github/Sedacoder/NLP/blob/main/sentiment_analysis_hotel_reviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
print("Hello World")

Hello World


In [ ]:
!pip install -U nltk
import nltk
import pandas as pd
import time
print("Loading data now.This could take a while depending on file size")
start = time.time()
df = pd.read_csv('Hotel_Reviews.csv')
end = time.time()
print("Loading took" + str(round(end - start, 2))+"seconds")

Loading data now.This could take a while depending on file size
Loading took4.56seconds


In [ ]:
df.columns

Index(['Hotel_Address', 'Additional_Number_of_Scoring', 'Review_Date',
       'Average_Score', 'Hotel_Name', 'Reviewer_Nationality',
       'Negative_Review', 'Review_Total_Negative_Word_Counts',
       'Total_Number_of_Reviews', 'Positive_Review',
       'Review_Total_Positive_Word_Counts',
       'Total_Number_of_Reviews_Reviewer_Has_Given', 'Reviewer_Score', 'Tags',
       'days_since_review', 'lat', 'lng'],
      dtype='object')

In [ ]:
print(df.shape)
nationality_freq = df["Reviewer_Nationality"].value_counts()

print("There are " + str(nationality_freq.size) + " different countries")
#nationality_freq.size counts the nations and gives the total number
#nationality_freq will print the first and last rows of the series.
print(nationality_freq)
print("\n ALL THE DATA")
print(nationality_freq.to_string())
#nationality_freq.to_string prints all the data


(515738, 17)
There are 227 different countries
Reviewer_Nationality
 United Kingdom               245246
 United States of America      35437
 Australia                     21686
 Ireland                       14827
 United Arab Emirates          10235
                               ...  
 Cape Verde                        1
 Northern Mariana Islands          1
 Tuvalu                            1
 Guinea                            1
 Palau                             1
Name: count, Length: 227, dtype: int64

 ALL THE DATA
Reviewer_Nationality
 United Kingdom                           245246
 United States of America                  35437
 Australia                                 21686
 Ireland                                   14827
 United Arab Emirates                      10235
 Saudi Arabia                               8951
 Netherlands                                8772
 Switzerland                                8678
 Germany                                    7941
 Canada  

In [ ]:
#.index[] will print the country at the placed index in the column
nationality_freq.index[2]

nationality_freq[1:11]#Prints the nations from index 1 to 11.


Reviewer_Nationality
 United States of America     35437
 Australia                    21686
 Ireland                      14827
 United Arab Emirates         10235
 Saudi Arabia                  8951
 Netherlands                   8772
 Switzerland                   8678
 Germany                       7941
 Canada                        7894
 France                        7296
Name: count, dtype: int64

In [ ]:
for nat in nationality_freq[:10].index:
  #Extracts all rows that match the criteria into a new dataframe
   nat_df = df[df["Reviewer_Nationality"] == nat]
   #Obtain the hotel frequency
   freq = nat_df["Hotel_Name"].value_counts()

print("The most reviewed hotel for " + str(nat).strip() + " was " + str(freq.index[0]) + " with " + str(freq[0]) + " reviews.")


The most reviewed hotel for Canada was St James Court A Taj Hotel London with 61 reviews.


In [ ]:
# First create a new dataframe based on the old one, removing the uneeded columns
hotel_freq_df = df.drop(["Hotel_Address", "Additional_Number_of_Scoring", "Review_Date", "Average_Score", "Reviewer_Nationality", "Negative_Review", "Review_Total_Negative_Word_Counts", "Positive_Review", "Review_Total_Positive_Word_Counts", "Total_Number_of_Reviews_Reviewer_Has_Given", "Reviewer_Score", "Tags", "days_since_review", "lat", "lng"], axis = 1)

# Group the rows by Hotel_Name, count them and put the result in a new column Total_Reviews_Found
hotel_freq_df['Total_Reviews_Found'] = hotel_freq_df.groupby('Hotel_Name').transform('count')

# Get rid of all the duplicated rows
hotel_freq_df = hotel_freq_df.drop_duplicates(subset = ["Hotel_Name"])
display(hotel_freq_df)

,Hotel_Name,Total_Number_of_Reviews,Total_Reviews_Found
0,Hotel Arena,1403,405
405,K K Hotel George,1831,566
971,Apex Temple Court Hotel,2619,1037
2008,The Park Grand London Paddington,4380,1770
3778,Monhotel Lounge SPA,171,35
...,...,...,...
511962,Suite Hotel 900 m zur Oper,3461,439
512401,Hotel Amadeus,717,144
512545,The Berkeley,232,100
512645,Holiday Inn London Kensington,5945,2768


In [ ]:
 def get_difference_review_avg(row):
  return row["Average_Score"] - row["Calc_Average_Score"]

#Creates Calc_Average_Score column
df['Calc_Average_Score'] = round(df.groupby('Hotel_Name').Reviewer_Score.transform('mean'), 1)

# Add a new column with the difference between the two average scores
df["Average_Score_Difference"] = df.apply(get_difference_review_avg, axis = 1)

# Create a df without all the duplicates of Hotel_Name (so only 1 row per hotel)
review_scores_df = df.drop_duplicates(subset = ["Hotel_Name"])


# Sort the dataframe to find the lowest and highest average score difference
review_scores_df = review_scores_df.sort_values(by=["Average_Score_Difference"])


display(review_scores_df[["Average_Score_Difference", "Average_Score", "Calc_Average_Score", "Hotel_Name"]])


,Average_Score_Difference,Average_Score,Calc_Average_Score,Hotel_Name
495945,-0.8,7.7,8.5,Best Western Hotel Astoria
111027,-0.7,8.8,9.5,Hotel Stendhal Place Vend me Paris MGallery by...
43688,-0.7,7.5,8.2,Mercure Paris Porte d Orleans
178253,-0.7,7.9,8.6,Renaissance Paris Vendome Hotel
218258,-0.5,7.0,7.5,Hotel Royal Elys es
...,...,...,...,...
151416,0.7,7.8,7.1,Best Western Allegro Nation
22189,0.8,7.1,6.3,Holiday Inn Paris Montparnasse Pasteur
250308,0.9,8.6,7.7,MARQUIS Faubourg St Honor Relais Ch teaux
68936,0.9,6.8,5.9,Villa Eugenie


In [ ]:
#Counting using lambda
start = time.time()
no_negative_reviews = df.apply(lambda x: True if x['Negative_Review'] == "No Negative" else False , axis=1)
print("Number of No Negative reviews: " + str(len(no_negative_reviews[no_negative_reviews == True].index)))

no_positive_reviews = df.apply(lambda x: True if x['Positive_Review'] == "No Positive" else False , axis=1)
print("Number of No Positive reviews: " + str(len(no_positive_reviews[no_positive_reviews == True].index)))

both_no_reviews = df.apply(lambda x: True if x['Negative_Review'] == "No Negative" and x['Positive_Review'] == "No Positive" else False , axis=1)
print("Number of both No Negative and No Positive reviews: " + str(len(both_no_reviews[both_no_reviews == True].index)))
end = time.time()
print("Lambdas took " + str(round(end - start, 2)) + " seconds")


Number of No Negative reviews: 127890
Number of No Positive reviews: 35946
Number of both No Negative and No Positive reviews: 127
Lambdas took 20.48 seconds


In [ ]:
# without lambdas (using a mixture of notations to show you can use both)
start = time.time()
no_negative_reviews = sum(df.Negative_Review == "No Negative")
print("Number of No Negative reviews: " + str(no_negative_reviews))

no_positive_reviews = sum(df["Positive_Review"] == "No Positive")
print("Number of No Positive reviews: " + str(no_positive_reviews))

both_no_reviews = sum((df.Negative_Review == "No Negative") & (df.Positive_Review == "No Positive"))
print("Number of both No Negative and No Positive reviews: " + str(both_no_reviews))

end = time.time()
print("Sum took " + str(round(end - start, 2)) + " seconds")


Number of No Negative reviews: 127890
Number of No Positive reviews: 35946
Number of both No Negative and No Positive reviews: 127
Sum took 0.94 seconds


Ayt. Now being done with this data exploration and performing operations on the data I just need to understand exactly what i have done and how I have done it.
Because this is essentially copied and pasted. Though  I have obviously read through the lesson. I am just forced to use the dataset given because I can't really get another one


In [ ]:
df.drop(["lat"], axis = 1, inplace=True)
df.drop(["lng"], axis = 1, inplace=True)

In [ ]:
def replace_address(row):
    if "Netherlands" in row["Hotel_Address"]:
        return "Amsterdam, Netherlands"
    elif "Barcelona" in row["Hotel_Address"]:
        return "Barcelona, Spain"
    elif "United Kingdom" in row["Hotel_Address"]:
        return "London, United Kingdom"
    elif "Milan" in row["Hotel_Address"]:
        return "Milan, Italy"
    elif "France" in row["Hotel_Address"]:
        return "Paris, France"
    elif "Vienna" in row["Hotel_Address"]:
        return "Vienna, Austria"

# Replace all the addresses with a shortened, more useful form
df["Hotel_Address"] = df.apply(replace_address, axis = 1)
# The sum of the value_counts() should add up to the total number of reviews
print(df["Hotel_Address"].value_counts())

display(df.groupby("Hotel_Address").agg({"Hotel_Name": "nunique"}))

Hotel_Address
London, United Kingdom    262301
Barcelona, Spain           60149
Paris, France              59928
Amsterdam, Netherlands     57214
Vienna, Austria            38939
Milan, Italy               37207
Name: count, dtype: int64


,Hotel_Name
Hotel_Address,
"Amsterdam, Netherlands",105
"Barcelona, Spain",211
"London, United Kingdom",400
"Milan, Italy",162
"Paris, France",458
"Vienna, Austria",158


In [ ]:
# Drop `Additional_Number_of_Scoring`
df.drop(["Additional_Number_of_Scoring"],axis= 1, inplace=True)
# Replace `Total_Number_of_Reviews` and `Average_Score` with our own calculated values
df.Total_Number_of_Reviews = df.groupby('Hotel_Name').transform('size')
df.Average_Score = round(df.groupby('Hotel_Name').Reviewer_Score.transform('mean'), 1)

In [ ]:
df.drop(["Review_Total_Negative_Word_Counts"], axis = 1, inplace=True)
df.drop(["Review_Total_Positive_Word_Counts"], axis = 1, inplace=True)
df.drop(["Review_Date"], axis = 1, inplace=True)
df.drop(["days_since_review"], axis = 1, inplace=True)


In [ ]:
# Remove opening and closing brackets
df.Tags = df.Tags.str.strip("[']")
# remove all quotes too
df.Tags = df.Tags.str.replace(" ', '", ",", regex = False)

In [ ]:
# Process the Tags into new columns
# The file Hotel_Reviews_Tags.py, identifies the most important tags
# Leisure trip, Couple, Solo traveler, Business trip, Group combined with Travelers with friends,
# Family with young children, Family with older children, With a pet
df["Leisure_trip"] = df.Tags.apply(lambda tag: 1 if "Leisure trip" in tag else 0)
df["Couple"] = df.Tags.apply(lambda tag: 1 if "Couple" in tag else 0)
df["Solo_traveler"] = df.Tags.apply(lambda tag: 1 if "Solo traveler" in tag else 0)
df["Business_trip"] = df.Tags.apply(lambda tag: 1 if "Business trip" in tag else 0)
df["Group"] = df.Tags.apply(lambda tag: 1 if "Group" in tag or "Travelers with friends" in tag else 0)
df["Family_with_young_children"] = df.Tags.apply(lambda tag: 1 if "Family with young children" in tag else 0)
df["Family_with_older_children"] = df.Tags.apply(lambda tag: 1 if "Family with older children" in tag else 0)
df["With_a_pet"] = df.Tags.apply(lambda tag: 1 if "With a pet" in tag else 0)

In [ ]:
df.head()

,Hotel_Address,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Total_Number_of_Reviews,Positive_Review,Total_Number_of_Reviews_Reviewer_Has_Given,Reviewer_Score,Tags,Calc_Average_Score,Average_Score_Difference,Leisure_trip,Couple,Solo_traveler,Business_trip,Group,Family_with_young_children,Family_with_older_children,With_a_pet
0,"Amsterdam, Netherlands",7.8,Hotel Arena,Russia,I am so angry that i made this post available...,405,Only the park outside of the hotel was beauti...,7,2.9,"Leisure trip, Couple, Duplex Double Room, Sta...",7.8,-0.1,1,1,0,0,0,0,0,0
1,"Amsterdam, Netherlands",7.8,Hotel Arena,Ireland,No Negative,405,No real complaints the hotel was great great ...,7,7.5,"Leisure trip, Couple, Duplex Double Room, Sta...",7.8,-0.1,1,1,0,0,0,0,0,0
2,"Amsterdam, Netherlands",7.8,Hotel Arena,Australia,Rooms are nice but for elderly a bit difficul...,405,Location was good and staff were ok It is cut...,9,7.1,"Leisure trip, Family with young children, Dup...",7.8,-0.1,1,0,0,0,0,1,0,0
3,"Amsterdam, Netherlands",7.8,Hotel Arena,United Kingdom,My room was dirty and I was afraid to walk ba...,405,Great location in nice surroundings the bar a...,1,3.8,"Leisure trip, Solo traveler, Duplex Double Ro...",7.8,-0.1,1,0,1,0,0,0,0,0
4,"Amsterdam, Netherlands",7.8,Hotel Arena,New Zealand,You When I booked with your company on line y...,405,Amazing location and building Romantic setting,3,6.7,"Leisure trip, Couple, Suite, Stayed 2 nights,...",7.8,-0.1,1,1,0,0,0,0,0,0


In [ ]:
# Saving new data file with calculated columns
print("Saving results to Hotel_Reviews_Filtered.csv")
df.to_csv('Hotel_Reviews_Filtered.csv', index = False)

Saving results to Hotel_Reviews_Filtered.csv
